<a href="https://colab.research.google.com/github/softstat/analysis/blob/main/Angle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive

/content/drive


In [3]:
cd MyDrive

/content/drive/MyDrive


In [4]:
cd Data

/content/drive/MyDrive/Data


In [5]:
ls

 angle_gait.csv              data.csv            weather.csv
'Clinical Assessment.xlsx'   titanic_train.csv


In [181]:
import pandas as pd
angle = pd.read_csv('angle_gait.csv')
clinical_assessment = pd.read_excel('Clinical Assessment.xlsx')

In [182]:
angle.drop('Unnamed: 0',axis=1,inplace=True)

In [183]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [184]:
def perform_pca(data, num_components):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data)
    pca = PCA(n_components=num_components)
    pca_result = pca.fit_transform(scaled_data)
    return pca_result

In [188]:
col = angle.columns
col = col.str.replace('[^\w\s]', '', regex=True)
angle.columns = col
num_components = 43

In [189]:
angle = angle.dropna()
na_sum = angle.isna().sum()

In [190]:
import random
random.seed(1)
total_angle_gait_pca = angle.copy()
total_angle_model = PCA(n_components=num_components)
total_angle_result = total_angle_model.fit_transform(total_angle_gait_pca)

In [191]:
total_angle_result

array([[-1.69715644e+02,  2.78757824e+01,  1.24143094e+02, ...,
        -3.92471603e+00,  2.51926619e+00,  8.50845056e-14],
       [-2.23333375e+02, -1.54750144e+01,  4.69437877e+01, ...,
        -4.22573239e-01,  3.29514125e+00,  8.50845056e-14],
       [ 6.22312161e+01, -1.43502742e+01,  5.24627212e+02, ...,
        -6.68759815e-01,  2.06298071e+00,  8.50845056e-14],
       ...,
       [ 2.16404983e+02,  2.36416445e+02, -8.90716272e+01, ...,
         2.55564438e-02,  7.50927346e+00,  8.50845056e-14],
       [-1.13801129e+01, -3.60015911e+01, -4.82583542e+01, ...,
         1.01146215e+01,  5.30972112e+00,  8.50845056e-14],
       [ 3.27822467e+01, -1.90162479e+02,  1.36776906e+02, ...,
         4.80209806e+00,  6.26140866e+00,  8.50845056e-14]])

In [192]:
import numpy as np
angle = np.array(angle)
non_angle = angle[:,0:2424]

In [193]:
random.seed(123)
non_angle_model = PCA(n_components=num_components)
non_angle_result = non_angle_model.fit_transform(non_angle)

In [194]:
non_angle_result.shape

(43, 43)

In [195]:
par_angle = angle[:,2525:4848]
random.seed(123)
par_angle_model = PCA(n_components=num_components)
par_angle_result = par_angle_model.fit_transform(par_angle)

In [196]:
score = clinical_assessment.iloc[:,9]

In [197]:
score = score.dropna()
del score[0]
del score[15]

In [198]:
y = []
for value in score:
    if value >= 45:
        y.append(1)
    else:
        y.append(0)
print(y)

[0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0]


## Non-paretic Side

In [225]:
df_non = pd.DataFrame(non_angle_result)

In [226]:
y = pd.DataFrame(y)

In [227]:
non = pd.concat([y,df_non],axis=1)

In [228]:
non.columns = ['PC'+ str(i) for i in range(0,44)]

In [229]:
colname = list(non.columns)
colname[0] = "score"
non.columns = colname
non.columns

Index(['score', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9',
       'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18',
       'PC19', 'PC20', 'PC21', 'PC22', 'PC23', 'PC24', 'PC25', 'PC26', 'PC27',
       'PC28', 'PC29', 'PC30', 'PC31', 'PC32', 'PC33', 'PC34', 'PC35', 'PC36',
       'PC37', 'PC38', 'PC39', 'PC40', 'PC41', 'PC42', 'PC43'],
      dtype='object')

In [238]:
X = non.loc[:,'PC1':'PC43']
y = non['score']
X_train , X_test , y_train , y_test  = train_test_split(X,y,test_size = 0.4,random_state=123)

In [239]:
k_best = SelectKBest(f_classif, k=30)  # 예제로 20개의 최상위 특성 선택
X_train_selected = k_best.fit_transform(X_train, y_train)
X_test_selected = k_best.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw


### logistic regression

In [240]:
model = LogisticRegression()
model.fit(X_train_selected, y_train)

LogisticRegression()

In [241]:
y_pred = model.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.72


In [242]:
cm = confusion_matrix(y_test,y_pred)
print(cm)

[[4 3]
 [2 9]]


In [243]:
from sklearn.metrics import f1_score
f1_score(y_test,y_pred)

0.7826086956521738

### Random Forest

In [244]:
from sklearn.metrics import f1_score
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_y_pred = rf_model.predict(X_test)
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
cm = confusion_matrix(y_test,y_pred)
print(cm)
f1_score = f1_score(y_test,y_pred)
print(f"f1_score:{f1_score:.2f}")

Accuracy: 0.61
[[ 0  7]
 [ 0 11]]
f1_score:0.76


### SVM

In [245]:
# SVM 모델
from sklearn.metrics import f1_score
svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train)
svm_y_pred = svm_model.predict(X_test)
y_pred = svm_model.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
cm = confusion_matrix(y_test,y_pred)
print(cm)
f1_score = f1_score(y_test,y_pred)
print(f"f1_score:{f1_score:.2f}")

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


ValueError: ignored

### XGBoost

In [ ]:
from sklearn.metrics import f1_score
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)
xgb_y_pred = xgb_model.predict(X_test)
y_pred = xgb_model.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
cm = confusion_matrix(y_test,y_pred)
print(cm)
f1_score = f1_score(y_test,y_pred)
print(f"f1_score:{f1_score:.2f}")

### LDA

In [ ]:
# LDA 모델
from sklearn.metrics import f1_score
lda_model = LDA()
lda_model.fit(X_train, y_train)
lda_y_pred = lda_model.predict(X_test)
y_pred = lda_model.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
cm = confusion_matrix(y_test,y_pred)
print(cm)
f1_score = f1_score(y_test,y_pred)
print(f"f1_score:{f1_score:.2f}")

## paretic-Side

In [ ]:
df_par = pd.DataFrame(par_angle_result)
y = pd.DataFrame(y)
par = pd.concat([y,df_par],axis=1)
par.columns = ['PC'+ str(i) for i in range(0,44)]
colname = list(par.columns)
colname[0] = "score"
par.columns = colname
par.columns

In [ ]:
random.seed(1)
X = par.loc[:,'PC1':'PC30']
y = par['score']
X_train , X_test , y_train , y_test  = train_test_split(X,y,test_size = 0.4,random_state=123)

In [ ]:
k_best = SelectKBest(f_classif, k=20)  # 예제로 10개의 최상위 특성 선택
X_train_selected = k_best.fit_transform(X_train, y_train)
X_test_selected = k_best.transform(X_test)

### logistic regression

In [ ]:
model = LogisticRegression()
model.fit(X_train_selected, y_train)

In [ ]:
y_pred = model.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
from sklearn.metrics import f1_score
cm = confusion_matrix(y_test,y_pred)
print(cm)
f1_score = f1_score(y_test,y_pred)
print(f"f1_score:{f1_score:.2f}")

### paretic RandomForest

In [ ]:
from sklearn.metrics import f1_score
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_y_pred = rf_model.predict(X_test)
y_pred = model.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
cm = confusion_matrix(y_test,y_pred)
print(cm)
f1_score = f1_score(y_test,y_pred)
print(f"f1_score:{f1_score:.2f}")

### paretic SVM

In [ ]:
# SVM 모델
from sklearn.metrics import f1_score
svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train)
svm_y_pred = svm_model.predict(X_test)
y_pred = model.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
cm = confusion_matrix(y_test,y_pred)
print(cm)
f1_score = f1_score(y_test,y_pred)
print(f"f1_score:{f1_score:.2f}")

### paretic XGBoost

In [ ]:
from sklearn.metrics import f1_score
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)
xgb_y_pred = xgb_model.predict(X_test)
y_pred = model.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
cm = confusion_matrix(y_test,y_pred)
print(cm)
f1_score = f1_score(y_test,y_pred)
print(f"f1_score:{f1_score:.2f}")

### paretic LDA

In [ ]:
# LDA 모델
from sklearn.metrics import f1_score
lda_model = LDA()
lda_model.fit(X_train, y_train)
lda_y_pred = lda_model.predict(X_test)
y_pred = model.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
cm = confusion_matrix(y_test,y_pred)
print(cm)
f1_score = f1_score(y_test,y_pred)
print(f"f1_score:{f1_score:.2f}")

cv=5 로 주고 해보고 feature selection 더 해보기